In [7]:
import sqlite3

romanian_texts = {}
moldavian_texts = {}

conn = sqlite3.connect('news.db')
c = conn.cursor()

c.execute('SELECT * FROM romania')
rows = c.fetchall()
for row in rows:
    if row[4] not in romanian_texts:
        romanian_texts[row[4]] = []
    romanian_texts[row[4]].append(row[5])
    
c.execute('SELECT * FROM moldova WHERE newspaper != "zugo"')
rows = c.fetchall()
for row in rows:
    if row[4] not in moldavian_texts:
        moldavian_texts[row[4]] = []
    moldavian_texts[row[4]].append(row[5])

conn.close()

In [20]:
all_texts = {"romana": '', "moldova": ''}

for key in romanian_texts:
    all_texts["romana"] += ' '.join(romanian_texts[key])

for key in moldavian_texts:
    all_texts["moldova"] += ' '.join(moldavian_texts[key])
    
all_texts['romana'] = all_texts['romana'][:1000000]
all_texts['moldova'] = all_texts['moldova'][:1000000]


In [ ]:
!python -m spacy download ro_core_news_md

In [144]:
import spacy

nlp = spacy.load('ro_core_news_md')

# Split text into 4-grams using spacy
def split_text(text):
    doc = nlp(text)
    # doc = [token for token in doc if not token.is_punct]
    return [doc[i:i+5] for i in range(len(doc)-4)]

all_grams = {'romana': split_text(all_texts['romana']), 'moldova': split_text(all_texts['moldova'])}

In [145]:
# Calculate the frequency of each 4-gram
from collections import Counter

all_grams_freq3 = {"romana": {}, "moldova": {}}

# Get 4 grams and add the 5th word with the frequency

for key in all_grams:
    for gram in all_grams[key]:
        gram4 = ''
        for i in range(4):
            gram4 += gram[i].text + ' '
        gram4 = gram4[:-1]
        if gram4 not in all_grams_freq3[key]:
            all_grams_freq3[key][gram4] = Counter()
        all_grams_freq3[key][gram4][gram[4].text] += 1

In [172]:
# Get a random 4-gram
import random

def get_random_4gram(language):
    return random.choice([x for x in all_grams_freq3[language].keys() if (x[0].isupper() or x[0] == '"' or x[0] == "'" or x[0] == '(') and '.' not in x and '?' not in x and '!' not in x])

# Get the most common 4-gram with 3 given words
def get_most_common_4gram(words, language):
    try:
        rv = all_grams_freq3[language][words].most_common(1)[0][0]   
        # print(1)
    except:
        rv = '.'
        
    return rv

# Generate a new text
def generate_text(language):
    text = get_random_4gram(language)
    text4 = [token for token in text.split(' ')]
    # text = ' '.join([token for token in text]))
    # print(all_grams_freq3[language][text].most_common(1))
    # print(text)
    for i in range(30):
        next_4gram = get_most_common_4gram(' '.join(text4), language)
        word = next_4gram.split(' ')[-1]
        
        text += ' ' + word
        text4 = text4[1:] + [word]
        
        if word in ['.', '?', '!']:
            break
        
    return text

txt_ro = generate_text('romana')
txt_md = generate_text('moldova')

print(f"Text generat automat Romania: {txt_ro}")
print(f"Text generat automat Moldova: {txt_md}")

Text generat automat Romania: PENTRU MAI MULTE REGIUNI DIN ROMÂNIA ) Un ciclon polar lovește România Așa cum spuneam , testul de astăzi are o dificultate ridicată și necesită ceva mai multă atenție și concentrare .
Text generat automat Moldova: Aici , Danemarca si Germania vor avea din start cate patru puncte , Romania si Polonia vor avea cate doua , iar Japonia si Serbia - nici cate unul .
